In [157]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "llama3.2:3b",
    temperature = 0,
    base_url='http://localhost:11434'
)

: 

# Exploração de Histórico e Rastreabilidade de Consultas usando LLMs

## Objetivo
Neste notebook, exploraremos os impactos do uso de histórico e rastreabilidade em consultas realizadas por meio de um modelo de linguagem grande (LLM). Utilizaremos metadados associados às interações para fornecer contexto e rastreabilidade, além de metadados gerados pelo próprio modelo para análise.

## Modelo Utilizado
Usaremos o modelo **Llama3.2:3B** como base, acessado via **Ollama**, uma ferramenta que facilita a integração e execução de modelos LLM localmente.

## Metodologia
1. **Histórico de Consultas**: Manteremos um log estruturado de todas as interações realizadas com o modelo.
2. **Rastreabilidade via Metadados**: Adicionaremos informações contextuais como usuário, data, e propósito da consulta, permitindo uma análise detalhada e rastreamento das interações.
3. **Metadados Gerados pelo Modelo**: Exploraremos como os metadados criados pelo modelo podem complementar o processo de rastreabilidade e análise.


## Resultado Esperado
- Análise do impacto da rastreabilidade nas consultas.
- Organização do histórico para facilitar a auditoria e reprodutibilidade.
- Identificação de padrões nos metadados gerados pelo modelo.

## Ferramentas e Recursos
- **Modelo**: Llama3.2:3B.
- **Framework**: Ollama.
- **Linguagem**: Python.


# Etapa 1: Pergunta Inicial ao LLM

Nesta etapa, faremos uma consulta direta ao modelo de linguagem (LLM) sem fornecer informações adicionais ou contextos baseados em documentos. O objetivo é avaliar a capacidade do modelo de responder à pergunta com base em seu treinamento e conhecimento prévio.

In [158]:
response = llm.invoke("Qual capitulo da lgpd trata do tratamento de dados pessoais pelo poder publico?")
print(response.content)

: 

: 


## Correção e Análise
O modelo não forneceu a resposta correta para a pergunta. De acordo com a Lei Geral de Proteção de Dados (LGPD), o capítulo que trata do tratamento de dados pessoais pelo poder público é o **Capítulo IV**, abrangendo os artigos que detalham as bases legais e os princípios aplicáveis ao tratamento de dados pelo Poder Público.

Agora, para melhorar o contexto e a precisão da resposta, iremos consumir o conteúdo do **PDF da LGPD** e enviá-lo como parte do contexto para a próxima consulta ao modelo.



In [9]:
import pdfplumber
from pytesseract import image_to_string

def extract_text_with_ocr(pdf_path):
    extracted_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:

            if page.extract_text():
                extracted_text.append(page.extract_text())
            else:
                page_image = page.to_image(resolution=300)
                image = page_image.original  
                text = image_to_string(image)
                extracted_text.append(text)
    return extracted_text




## Objetivo
O objetivo desta etapa é extrair o texto completo do **PDF da LGPD** para que possamos usá-lo como contexto na próxima consulta ao modelo. 

## Método Utilizado
1. **PDFPlumber**: Usado para extrair texto diretamente de páginas do PDF.
2. **OCR (Reconhecimento Óptico de Caracteres)**: Caso o texto de uma página não seja extraído diretamente (por exemplo, devido a imagens), aplicamos o OCR usando o `pytesseract`.


In [ ]:
pdf_path = "data/lgpd.pdf"
text = extract_text_with_ocr(pdf_path)



 Integração com LangChain, FAISS e Modelo Ollama

## Objetivo
Este código implementa uma pipeline de **Retrieval-Augmented Generation (RAG)**, utilizando **LangChain**, **FAISS** e o modelo **Ollama**. O objetivo é fornecer respostas contextuais baseadas em documentos previamente indexados.

## **1. LangChain**
O LangChain é uma biblioteca projetada para facilitar a integração de modelos de linguagem com ferramentas externas, como bases de dados, vetores de busca e fluxos de trabalho complexos.

### Componentes Utilizados:
- **`langchain.schema.Document`**: Representa um documento estruturado com conteúdo textual que pode ser utilizado em pipelines de recuperação de informações.
- **`langchain.schema.HumanMessage`**: Utilizado para encapsular mensagens enviadas por um usuário humano para interação com modelos de linguagem.

---

## **2. HuggingFaceEmbeddings**
Esta tecnologia é utilizada para converter textos em representações vetoriais (embeddings) que podem ser usadas para buscas semânticas e recuperação de informações.

- **Modelo Utilizado**: `all-MiniLM-L6-v2`
  - Este modelo, baseado no framework HuggingFace, é otimizado para gerar embeddings de alta qualidade em tarefas de similaridade semântica e recuperação de informações.
- **Função**: Permitir a indexação de textos em um espaço vetorial, tornando possível a busca por documentos semelhantes com base em consultas.

---

## **3. FAISS (Facebook AI Similarity Search)**
FAISS é uma biblioteca altamente otimizada para busca de similaridade em grandes coleções de dados vetoriais.

### Por que FAISS?
- **Eficiência**: Projetado para realizar buscas rápidas e escaláveis em dados vetoriais.
- **Integração**: Fácil integração com embeddings gerados pelo HuggingFace ou outros frameworks.
- **Função no Código**:
  - Armazena os embeddings dos documentos.
  - Permite recuperar os documentos mais relevantes com base na similaridade entre o texto da consulta e os documentos indexados.

---

In [14]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document, HumanMessage
from langchain.vectorstores import FAISS



def create_documents(texts):
    if isinstance(texts, list):
        documents = [Document(page_content=text) for text in texts]
    else:
        documents = [Document(page_content=texts)]
    return documents

def create_faiss_vectorstore(documents):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore

def generate_response_with_ollama(vectorstore, query):
    retriever = vectorstore.as_retriever()
    retrieved_docs = retriever.invoke(query)
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    
    print("Contexto Recuperado:")
    print(context[:200])
    
    prompt = f"Contexto: {context}\n\nPergunta: {query}\nResposta:"
    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content.strip()

def rag_pipeline_with_ollama(texts, query):
    documents = create_documents(texts)
    vectorstore = create_faiss_vectorstore(documents)
    response = generate_response_with_ollama(vectorstore, query)
    return response




In [ ]:
query = "Qual capítulo da LGPD trata do tratamento de dados pessoais pelo poder público?"
response = rag_pipeline_with_ollama(text, query)

# Exibir resposta gerada
print("\nResposta Gerada:", response)

## Análise da Resposta
Após incluir o texto da **Lei Geral de Proteção de Dados (LGPD)** como contexto, o modelo conseguiu gerar uma resposta **correta e precisa**. O **Capítulo IV** é, de fato, o capítulo da LGPD que trata do tratamento de dados pessoais pelo poder público.

---

## Observação
Este resultado demonstra que o fornecimento de um **contexto relevante** (neste caso, o texto da lei) melhora significativamente a precisão das respostas geradas pelo modelo. Inicialmente, sem o contexto, a resposta era incorreta.

---

## Benefício do Contexto Adicional
A inclusão do texto da LGPD como contexto foi essencial para:
1. **Proporcionar ao modelo informações diretas e relevantes**.
2. **Garantir que a resposta fosse fundamentada em uma fonte confiável**.

## Proximos passos será o de usar multiplos documentos


In [16]:
import os
import pdfplumber
from pytesseract import image_to_string
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_ollama import ChatOllama


In [17]:
def extract_text_with_ocr(pdf_path):

    extracted_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            if page.extract_text():
                extracted_text.append(page.extract_text())
            else:
                page_image = page.to_image(resolution=300)
                image = page_image.original
                text = image_to_string(image, lang="por")
                extracted_text.append(text)
    return "\n".join(extracted_text)


In [18]:
def load_pdf_documents(pdf_paths):

    all_documents = []
    for path in pdf_paths:
        print(f"Carregando: {path}")
        try:
            # Extract and consolidate text from the PDF
            text = extract_text_with_ocr(path)
            filename = os.path.basename(path)
            document = Document(
                page_content=text,
                metadata={"source": filename}  # Metadata includes the file name
            )
            all_documents.append(document)
            print(f"Documento consolidado para {filename}")
        except Exception as e:
            print(f"Erro ao carregar {path}: {e}")

    if not all_documents:
        raise ValueError("Nenhum documento foi carregado. Verifique os caminhos dos PDFs.")
    
    print(f"Total de documentos carregados: {len(all_documents)}")
    return all_documents


In [19]:
def create_vectorstore(documents):

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore


In [20]:
def generate_response(query, vectorstore):

    docs = vectorstore.similarity_search(query, k=5)
    context = "\n\n".join([
        f"Source: {doc.metadata.get('source', 'Unknown')}\nContent: {doc.page_content}" for doc in docs
    ])
    

    prompt = f"Contexto: {context}\n\nPergunta: {query}\nResposta:"
    response = llm.invoke([prompt])
    
    output = {
        "question": query,
        "context": context[:500],
        "response": response.content.strip(),
        "sources": [doc.metadata.get("source", "Unknown") for doc in docs]
    }
    return output


In [21]:
def prepare_vectorstore(pdf_paths):

    documents = load_pdf_documents(pdf_paths)
    vectorstore = create_vectorstore(documents)
    print("Vectorstore criado com sucesso.")
    return vectorstore


In [22]:
def ask_question(query, vectorstore):

    response = generate_response(query, vectorstore)
    return response


In [ ]:
pdf_files = ["data/lgpd.pdf", "data/codigo_civil.pdf"]
vectorstore = prepare_vectorstore(pdf_files)

In [ ]:
query1 = "definicao lei geral de protecao de dados"
response1 = ask_question(query1, vectorstore)
print("Pergunta:", response1["question"])

print("-------------------------------------------------------------------------------------------------------")
print("\nResposta Gerada:")
print(response1["response"])

print("-------------------------------------------------------------------------------------------------------")

print("Contexto:", response1["context"])
print("\nFontes:")
for source in set(response1["sources"]):
    print("-", source)

print("-------------------------------------------------------------------------------------------------------")
print("\n")


In [ ]:
query2 = "capitulo das perdas e danos?"
response2 = ask_question(query2, vectorstore)
print("Pergunta:", response2["question"])

print("-------------------------------------------------------------------------------------------------------")
print("\nResposta Gerada:")
print(response2["response"])

print("-------------------------------------------------------------------------------------------------------")

print("Contexto:", response2["context"])
print("\nFontes:")
for source in set(response2["sources"]):
    print("-", source)

print("-------------------------------------------------------------------------------------------------------")
print("\n")


## Observação Inicial
Recuperamos as fontes utilizadas pelo sistema para responder a perguntas. No entanto, notamos que elas não estão **bem distribuídas**. 

### Problema Identificado
Ao fazer perguntas específicas sobre a **LGPD**, o sistema consultou fontes não relacionadas, como o **Código Civil** (`codigo_civil.pdf`). Isso indica que:
1. O sistema de recuperação de documentos está incluindo fontes irrelevantes.
2. As respostas geradas podem ser influenciadas por documentos que não são pertinentes ao tema consultado.

---

## Solução Proposta: Uso de Chunks
Para melhorar a precisão e relevância das fontes utilizadas, vamos implementar o **uso de chunks**. Essa abordagem permite dividir documentos em blocos menores, facilitando a recuperação de partes mais específicas e relevantes.

### Benefícios do Uso de Chunks
1. **Maior Precisão**: Reduz a probabilidade de incluir documentos irrelevantes ao restringir a busca a trechos específicos.
2. **Contexto Focado**: Aumenta a relevância do contexto recuperado para responder a perguntas específicas.
3. **Melhor Gerenciamento de Fontes**: Garante que as respostas sejam fundamentadas em documentos diretamente relacionados ao tema.


In [28]:
def split_text_into_chunks(text, chunk_size=1000, overlap=200):

    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap if end - overlap > start else end
    return chunks


In [29]:
def load_pdf_documents_with_chunks(pdf_paths, chunk_size=1000, overlap=200):

    all_documents = []
    for path in pdf_paths:
        print(f"Carregando: {path}")
        try:
            text = extract_text_with_ocr(path)
            filename = os.path.basename(path)
            chunks = split_text_into_chunks(text, chunk_size, overlap)
            for i, chunk in enumerate(chunks):
                document = Document(
                    page_content=chunk,
                    metadata={"source": filename, "chunk_index": i}
                )
                all_documents.append(document)
            print(f"{len(chunks)} chunks criados para {filename}")
        except Exception as e:
            print(f"Erro ao carregar {path}: {e}")

    if not all_documents:
        raise ValueError("Nenhum documento foi carregado. Verifique os caminhos dos PDFs.")
    
    print(f"Total de chunks carregados: {len(all_documents)}")
    return all_documents


In [30]:
def prepare_vectorstore_with_chunks(pdf_paths, chunk_size=1000, overlap=200):

    documents = load_pdf_documents_with_chunks(pdf_paths, chunk_size, overlap)
    vectorstore = create_vectorstore(documents)
    print("Vectorstore criado com chunks.")
    return vectorstore


In [ ]:
# Carregar documentos, dividir em chunks e criar vetorstore
pdf_files = ["data/lgpd.pdf", "data/codigo_civil.pdf"]
vectorstore_with_chunks = prepare_vectorstore_with_chunks(pdf_files, chunk_size=1000, overlap=200)


# Reexecução de Perguntas Após a Divisão em Chunks

## Objetivo
Após implementar a divisão dos documentos em chunks, vamos refazer as mesmas perguntas para avaliar se:
1. A recuperação de contexto está mais precisa.
2. Fontes irrelevantes, como o **Código Civil**, não estão mais sendo consultadas em perguntas sobre a **LGPD**.
3. As respostas geradas pelo modelo são mais relevantes e contextualizadas.

---

## Metodologia
1. **Pipeline Atualizada**: Utilizar a pipeline com o vetorstore criado a partir dos documentos divididos em chunks.
2. **Análise das Respostas**:
   - Verificar se os chunks recuperados pertencem às fontes esperadas.
   - Avaliar se as respostas geradas estão em conformidade com a pergunta.
3. **Registro das Fontes**:
   - Analisar as fontes consultadas para garantir que apenas documentos relevantes foram utilizados.

---

## Resultados Esperados
1. Respostas mais precisas e contextualizadas, diretamente relacionadas aos documentos corretos.
2. Consultas à **LGPD** limitadas aos chunks de `lgpd.pdf`, eliminando interferências de fontes irrelevantes como `codigo_civil.pdf`.
3. Melhor rastreabilidade das fontes devido aos metadados incluídos em cada chunk.

---

In [ ]:
# Fazer uma pergunta
query1 = "definicao lei geral de protecao de dados"
response1 = ask_question(query1, vectorstore_with_chunks)

print("Pergunta:", response1["question"])
print("\nResposta Gerada:")
print(response1["response"])



print("\nFontes:")
for source in set(response1["sources"]):
    print("-", source)

print("\n")
print("-------------------------------------------------------------------------------------------------------")
print("\n")



In [ ]:
query2 = "capitulo das perdas e danos?"
response2 = ask_question(query2, vectorstore_with_chunks)

print("Pergunta:", response2["question"])
print("\nResposta Gerada:")
print(response2["response"])

print("\nFontes:")
for source in set(response2["sources"]):
    print("-", source)


# Resultados Após a Implementação dos Chunks

## Observação
Após dividir os documentos em **chunks**, refizemos as perguntas utilizando a nova pipeline. Com essa abordagem, conseguimos:

1. **Recuperar as Fontes Corretamente**:
   - As perguntas sobre a **LGPD** consultaram apenas os chunks relevantes de `lgpd.pdf`.
   - Fontes irrelevantes, como o `codigo_civil.pdf`, não foram mais incluídas no contexto.

2. **Respostas Mais Assertivas**:
   - As respostas geradas pelo modelo foram mais precisas e alinhadas às perguntas feitas.
   - A relevância e o contexto melhoraram significativamente devido ao uso de chunks menores e bem definidos.

---

## Benefícios Identificados
1. **Melhor Foco no Contexto**:
   - O sistema agora prioriza trechos específicos dos documentos, reduzindo a interferência de fontes não relacionadas.
2. **Aumento na Precisão**:
   - Respostas mais confiáveis e fundamentadas, com base em documentos diretamente relevantes ao tema da pergunta.
3. **Rastreabilidade**:
   - Cada resposta pode ser associada aos chunks e documentos específicos usados para gerar o contexto.

---

# Implementação de Histórico na Interação com o Modelo

## Objetivo
Adicionar um histórico de conversas para que o modelo possa basear suas respostas em interações anteriores, fornecendo um contexto mais rico e um feedback mais assertivo.

---

In [39]:
def summarize_history_with_previous(history):
    """
    Inclua todo o histórico, destacando explicitamente a interação mais recente.
    """
    if not history:
        return "Histórico vazio."

    # Última interação (pergunta e resposta anterior)
    last_interaction = f"Pergunta Anterior: {history[-1][0]}\nResposta Anterior: {history[-1][1]}"
    
    # Outras interações (exceto a última)
    previous_interactions = "\n".join([
        f"Pergunta: {q}\nResposta: {r}" for q, r in history[:-1]
    ])
    
    # Combinar tudo
    return f"{previous_interactions}\n\n{last_interaction}" if previous_interactions else last_interaction




In [40]:
def format_prompt_with_history(query, context, history):

    # Incluir o histórico completo
    full_history = summarize_history_with_previous(history)
    
    # Destacar a última interação diretamente no contexto
    if history:
        last_question, last_answer = history[-1]
        last_interaction_context = f"Baseando-se na última interação:\nPergunta: {last_question}\nResposta: {last_answer}\n"
    else:
        last_interaction_context = ""

    # Construir o prompt
    prompt = f"""
    Você é um assistente jurídico especializado no Código Civil brasileiro.

    Última Interação:
    {last_interaction_context}

    Histórico de Conversa:
    {full_history}

    Trechos relevantes dos documentos:
    {context}

    Nova Pergunta:
    {query}

    Resposta:
    """
    return prompt


In [41]:

def generate_response_with_history(query, vectorstore, history):

    docs = vectorstore.similarity_search(query, k=5)
    
    context = "\n\n".join([
        f"Fonte: {doc.metadata.get('source', 'Desconhecido')}\nTrecho: {doc.page_content[:300]}" for doc in docs
    ])
    
    
    prompt = format_prompt_with_history(query, context, history)
    
    response = llm.invoke([prompt])
    
    history.append((query, response.content.strip()))
    
    output = {
        "question": query,
        "response": response.content.strip(),
        "sources": [doc.metadata.get("source", "Desconhecido") for doc in docs],
        "context_used": context  
    }
    return output




In [42]:

def ask_question_with_history(query, vectorstore, history):

    return generate_response_with_history(query, vectorstore, history)

In [ ]:
conversation_history = []

response1 = ask_question_with_history(
    "Quais são os capítulos e seções do Livro II do Código Civil?", 
    vectorstore_with_chunks, 
    conversation_history
)

print("\nResposta Gerada:")
print(response1["response"])
print(response1["context_used"])
print("\nFontes:")
for source in set(response1["sources"]):
    print("-", source)



## Análise da Resposta
A resposta gerada está **incorreta**. Os capítulos e seções apresentados não correspondem ao conteúdo real do **Livro II do Código Civil brasileiro**.

---

## Próxima Ação: Passar a Definição Correta no Próximo Input
Para corrigir o erro, vamos:
1. **Inserir a definição correta** dos capítulos e seções do Livro II como parte do próximo input.
2. **Atualizar o histórico** para que o modelo possa melhorar sua resposta com base no contexto correto.


In [ ]:
response2 = ask_question_with_history(
    '''Os capítulos do Livro II do Código Civil são Livro II – Dos Bens
Título Único – Das Diferentes Classes de Bens

Capítulo I – Dos Bens Considerados em Si Mesmos
Seção I – Dos Bens Imóveis
Seção II – Dos Bens Móveis
Seção III – Dos Bens Fungíveis e Consumíveis
Seção IV – Dos Bens Divisíveis
Seção V – Dos Bens Singulares e Coletivos
Capítulo II – Dos Bens Reciprocamente Considerados
Capítulo III – Dos Bens Públicos''', 
    vectorstore_with_chunks, 
    conversation_history
)

print("\nResposta Gerada:")
print(response2["response"])
print("\nFontes:")
for source in set(response2["sources"]):
    print("-", source)



# Reexecução da Pergunta com Feedback Fornecido

## Contexto
Após identificar que a resposta gerada anteriormente estava **incorreta**, fornecemos ao modelo o feedback necessário com a definição correta dos capítulos e seções do **Livro II do Código Civil**.

---

## Próxima Ação
Agora, faremos a mesma pergunta novamente para verificar se o modelo:
1. **Incorporou o feedback fornecido**.
2. **Gera uma resposta correta** baseada no novo contexto.

---


## Próximos Passos
1. **Executar a pergunta novamente**.
2. **Analisar a resposta gerada** para validar se ela corresponde ao contexto fornecido.
3. Documentar os resultados e, se necessário, continuar ajustando o modelo.


In [ ]:
response3 = ask_question_with_history(
    "Quais são os capítulos e seções do Livro II do Código Civil?", 
    vectorstore_with_chunks, 
    conversation_history
)

print("\nResposta Gerada:")
print(response3["response"])
print("\nFontes:")
for source in set(response3["sources"]):
    print("-", source)


# Uso do Histórico e Introdução ao Cross-Encoder

## 1. Histórico Utilizado com Sucesso
O uso do histórico na interação demonstrou ser eficaz para melhorar a precisão das respostas. Após fornecer o feedback necessário ao modelo e reutilizar o histórico, conseguimos obter a **resposta correta** para a pergunta:

**Pergunta**: *"Quais são os capítulos e seções do Livro II do Código Civil?"*

**Resposta Gerada**:  
- **Livro II – Dos Bens**  
- **Título Único – Das Diferentes Classes de Bens**

### Benefícios do Histórico
1. **Aprendizado Contínuo**: O modelo ajustou suas respostas com base no feedback e nas interações anteriores.
2. **Melhoria na Precisão**: As respostas passaram a refletir melhor o contexto fornecido.
3. **Contexto Rico**: O histórico permitiu ao modelo considerar informações adicionais e construir respostas mais completas.

---

## 2. Hipótese: Uso de Cross-Encoder para Melhorar a Rastreabilidade
Podemos explorar o uso de um **cross-encoder** para refinar ainda mais o sistema. 

### O que é um Cross-Encoder?
O cross-encoder é uma técnica de aprendizado profundo que:
1. **Atribui pontuações de similaridade** diretamente entre a consulta e os trechos de texto recuperados.
2. **Reavalia o contexto de maneira mais precisa** ao considerar todas as interações entre o texto e a consulta.

### Benefícios Esperados do Cross-Encoder
1. **Melhor Rastreabilidade**:
   - O cross-encoder pode identificar com maior precisão os trechos mais relevantes, permitindo que o modelo utilize apenas os dados mais confiáveis.
2. **Contexto Otimizado**:
   - Refinando a seleção de trechos, o modelo terá um contexto mais relevante para gerar respostas.
3. **Respostas Mais Precisas**:
   - O uso de pontuações de similaridade permite ao sistema priorizar trechos que respondam diretamente à consulta, minimizando a interferência de informações secundárias.

### Próximos Passos
1. **Implementar o Cross-Encoder**:
   - Integrar o modelo de cross-encoder para reavaliar os trechos recuperados antes de passá-los ao modelo.
2. **Validar Hipótese**:
   - Testar o impacto do cross-encoder na rastreabilidade e precisão das respostas.
3. **Comparação com Resultados Anteriores**:
   - Analisar as melhorias introduzidas pelo cross-encoder em relação ao uso do histórico sozinho.

---

## Conclusão
O uso do histórico melhorou significativamente as respostas do modelo, mas a introdução de um cross-encoder pode ser uma solução adicional para refinar ainda mais a rastreabilidade e precisão do sistema.


In [52]:
from typing import List
from sentence_transformers import CrossEncoder


def rerank_documents_with_crossencoder(query: str, docs: List, model_name: str = "cross-encoder/ms-marco-MiniLM-L-6-v2") -> List:
    """
    Reorganiza os documentos usando um modelo CrossEncoder com base na relevância para a consulta.
    """
    if not docs:
        return []

    # Inicializar o modelo CrossEncoder
    cross_encoder = CrossEncoder(model_name)

    # Criar pares de (query, document) para o modelo
    query_doc_pairs = [(query, doc.page_content) for doc in docs]

    # Obter scores de relevância
    scores = cross_encoder.predict(query_doc_pairs)

    # Ordenar os documentos pelo score em ordem decrescente
    sorted_docs = [doc for _, doc in sorted(zip(scores, docs), key=lambda x: x[0], reverse=True)]

    return sorted_docs



def generate_response_with_rerank(query: str, vectorstore, history: List) -> dict:
    """
    Gera uma resposta utilizando re-ranking dos documentos mais relevantes para a consulta.
    """
    # Buscar mais documentos relevantes no Vectorstore
    initial_docs = vectorstore.similarity_search(query, k=10)  # Aumentado para considerar mais documentos

    if not initial_docs:
        return {
            "question": query,
            "response": "Nenhum documento relevante encontrado para a consulta.",
            "sources": [],
            "context_used": ""
        }

    # Aplicar re-ranking nos documentos retornados
    reranked_docs = rerank_documents_with_crossencoder(query, initial_docs)

    # Selecionar os top 5 documentos após o re-ranking
  

    # Construir o contexto a partir dos documentos selecionados
    context = "\n\n".join([
        f"Fonte: {doc.metadata.get('source', 'Desconhecido')}\nTrecho: {doc.page_content}" for doc in reranked_docs
    ])

    # Gerar o prompt com o histórico e contexto
    #prompt 

    prompt = format_prompt_with_history(query, context, history)
    # Obter a resposta do modelo LLM
    response = llm.invoke([prompt])

    # Adicionar a interação ao histórico
    history.append((query, response.content.strip()))

    # Formatar a saída
    output = {
        "question": query,
        "response": response.content.strip(),
        "sources": [doc.metadata.get("source", "Desconhecido") for doc in reranked_docs],
        "context_used": context
    }
    return output

def ask_question_with_rerank(query: str, vectorstore, history: List) -> dict:
    """
    Wrapper para gerar respostas com re-ranking utilizando o CrossEncoder.
    """
    return generate_response_with_rerank(query, vectorstore, history)


In [ ]:
conversation_history = []

response1 = ask_question_with_rerank(
    "Quais são os capítulos e seções do Livro II do Código Civil brasileiro?", 
    vectorstore_with_chunks, 
    conversation_history
    
)

print("\nResposta Gerada:")
print(response1["response"])
print(response1["context_used"])
print("\nFontes:")
for source in set(response1["sources"]):
    print("-", source)



# Resultados do Uso de Rerank na Recuperação de Contexto

## Observação
Com a introdução do **rerank** no pipeline. A técnica conseguiu priorizar trechos que mencionam mais diretamente a **keyword "Livro II"**, resultando em um contexto mais relevante para a pergunta.

---


## Limitações Identificadas
Apesar da melhoria no contexto recuperado, a **resposta gerada pelo modelo ainda está incorreta**. Isso pode indicar:
1. **Interpretação Limitada do Modelo**:
   - O modelo não está aproveitando plenamente o contexto priorizado pelo rerank.
2. **Necessidade de Ajustes no Contexto**:
   - O contexto, embora relevante, pode não estar detalhado o suficiente para auxiliar o modelo.



In [ ]:
response2 = ask_question_with_rerank(
    '''Os capítulos do Livro II do Código Civil são Livro II – Dos Bens
Título Único – Das Diferentes Classes de Bens

Capítulo I – Dos Bens Considerados em Si Mesmos
Seção I – Dos Bens Imóveis
Seção II – Dos Bens Móveis
Seção III – Dos Bens Fungíveis e Consumíveis
Seção IV – Dos Bens Divisíveis
Seção V – Dos Bens Singulares e Coletivos
Capítulo II – Dos Bens Reciprocamente Considerados
Capítulo III – Dos Bens Públicos''', 
    vectorstore_with_chunks, 
    conversation_history
)

print("\nResposta Gerada:")
print(response2["response"])
print("\nFontes:")
for source in set(response2["sources"]):
    print("-", source)




In [ ]:
response3 = ask_question_with_rerank(
    "Quais são os capítulos e seções do Livro II do Código Civil?", 
    vectorstore_with_chunks, 
    conversation_history
)

print("\nResposta Gerada:")
print(response3["response"])
print("\nFontes:")
for source in set(response3["sources"]):
    print("-", source)

# Análise: Histórico e Rerank

## Observação Geral
Embora tenhamos introduzido o uso do histórico e do rerank para melhorar a recuperação de contexto e a precisão das respostas, o modelo apresentou limitações em interações mais longas e iterativas. Na terceira pergunta, mesmo após o feedback fornecido, o modelo **não conseguiu encontrar a resposta correta**.

---



# Introdução ao Uso de Metadados na Recuperação de Documentos

## Objetivo
Melhorar a rastreabilidade e precisão na recuperação de informações ao:
1. **Incluir metadados** como Livro, Capítulo e Seção em cada chunk extraído dos documentos.
2. Estruturar os chunks de forma mais granular, associando cada parte do texto aos seus respectivos metadados.

---
## Hipotese
1. Preparar um vetorstore utilizando documentos com metadados para consultas mais precisas.
---

## Beneficio
1. Facilitar a rastreabilidade, associando cada chunk aos metadados contextuais.

In [63]:
import re

def split_chunks_v2(text, chunk_size=1000, overlap=200):

    # Padrões para detectar Livro, Capítulo e Seção (case-insensitive)
    livro_pattern = re.compile(r"(Livro\s+[IVXLCDM\d]+\s*.*?)\s*(?=\n|$)", re.IGNORECASE)
    capitulo_pattern = re.compile(r"(Capítulo\s+[IVXLCDM\d]+\s*.*?)\s*(?=\n|$)", re.IGNORECASE)
    secao_pattern = re.compile(r"(Seção\s+[IVXLCDM\d]+\s*.*?)\s*(?=\n|$)", re.IGNORECASE)

    # Variáveis para acompanhar os metadados atuais
    current_livro = None
    current_capitulo = None
    current_secao = None

    # Lista de chunks e variáveis de controle
    chunks = []
    current_chunk = []
    current_size = 0

    def save_chunk_and_reset():
        nonlocal current_chunk, current_size
        if current_chunk:
            chunks.append({
                "content": "\n".join(current_chunk),
                "metadata": {
                    "livro": current_livro,
                    "capítulo": current_capitulo,
                    "seção": current_secao
                }
            })
            # Reiniciar mantendo o overlap
            current_chunk = current_chunk[-overlap:]
            current_size = sum(len(line) for line in current_chunk)

    # Processar linhas do texto
    for line in text.split("\n"):
        # Detectar mudanças nos metadados
        if livro_match := livro_pattern.match(line):
            save_chunk_and_reset()
            current_livro = livro_match.group(1)

        if capitulo_match := capitulo_pattern.match(line):
            save_chunk_and_reset()
            current_capitulo = capitulo_match.group(1)

        if secao_match := secao_pattern.match(line):
            save_chunk_and_reset()
            current_secao = secao_match.group(1)

        # Adicionar linha ao chunk atual
        current_chunk.append(line)
        current_size += len(line)

        # Criar chunk se atingir o tamanho definido
        if current_size >= chunk_size:
            save_chunk_and_reset()

    # Adicionar o último chunk
    save_chunk_and_reset()

    return chunks


In [67]:
import pdfplumber
from langchain.docstore.document import Document

def load_pdf_documents_v2(pdf_paths, chunk_size=1000, overlap=200):

    documents = []
    
    for pdf_path in pdf_paths:
        try:
            # Abrir o PDF
            with pdfplumber.open(pdf_path) as pdf:
                print(f"Processando arquivo: {pdf_path}")
                for page_number, page in enumerate(pdf.pages, start=1):
                    # Extrair texto da página
                    text = page.extract_text()
                    
                    if text:


                        # Dividir o texto em chunks
                        chunks = split_chunks_v2(text, chunk_size, overlap)
                        
                        if not chunks:
                            continue
                        
                        # Adicionar chunks à lista de documentos
                        for chunk in chunks:
                            documents.append(Document(
                                page_content=chunk["content"],
                                metadata={
                                    "source": pdf_path,
                                    "page": page_number,
                                    **chunk["metadata"]  # Inclui metadados como Livro, Capítulo, Seção
                                }
                            ))
                    else:
                            pass
        except Exception as e:
            print(f"Erro ao processar o arquivo {pdf_path}: {e}")
    
    if not documents:
        print("Nenhum documento foi carregado. Verifique os arquivos PDF ou os métodos de extração.")
    else:
        print(f"Carregados {len(documents)} documentos do total de {len(pdf_paths)} arquivos PDF.")

    return documents


In [68]:
def prepare_vectorstore_with_chunks_v2(pdf_paths, chunk_size=2000, overlap=300):
    # Carregar documentos estruturados com resumos
    documents = load_pdf_documents_v2(pdf_paths, chunk_size, overlap)

    # Criar vetorstore
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(documents, embeddings)
    
    return [vectorstore, documents]

In [ ]:
vectorstore_with_chunks_v, documents = prepare_vectorstore_with_chunks_v2(pdf_files, chunk_size=2000, overlap=300)

In [ ]:


def ask_question_with_history_v2(question, vectorstore, conversation_history):

    from langchain.chains import ConversationalRetrievalChain
    from langchain.chains.question_answering import load_qa_chain
    from langchain.chat_models import ChatOpenAI


    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        return_source_documents=True  # Garante que as fontes sejam retornadas
    )

    result = qa_chain({"question": question, "chat_history": conversation_history})

    conversation_history.append((question, result["answer"]))

    metadata_list = []
    for doc in result["source_documents"]:
        metadata = doc.metadata
        metadata_list.append({
            "livro": metadata.get("livro"),
            "capítulo": metadata.get("capítulo"),
            "seção": metadata.get("seção"),
            "content": doc.page_content  
        })

    return {
        "response": result["answer"],
        "context_used": metadata_list,
    }


# Exemplo de uso
conversation_history = []
response1 = ask_question_with_history_v2(
    "Perdas e danos",
    vectorstore_with_chunks_v,
    conversation_history
)

# Exibir resposta e contexto
print("\nResposta Gerada:")
print(response1["response"])

print("-------------------------------------------------------------------------------------------------------")

# print("Contexto utilizado:")
# for context in response1["context_used"]:
#     print(f"Livro: {context['livro']}")
#     print(f"Capítulo: {context['capítulo']}")
#     print(f"Seção: {context['seção']}")
#     print("Conteúdo:\n", context["content"])
#     print("\n---\n")
print("Primeiro Contexto Utilizado:")
if response1["context_used"]:
    first_context = response1["context_used"][0]
    print(f"Livro: {first_context['livro']}")
    print(f"Capítulo: {first_context['capítulo']}")
    print(f"Seção: {first_context['seção']}")
    print("Conteúdo:\n", first_context["content"])
else:
    print("Nenhum contexto foi utilizado.")


---

## Análise

### Metadados
- **Capítulo**: O modelo identificou corretamente o **Capítulo III – Das Perdas e Danos** como relevante.
- **Livro e Seção**: Não foram detectados metadados específicos, mas o capítulo principal foi recuperado com sucesso.

### Contexto
- O conteúdo recuperado cita diretamente os artigos legais que tratam de perdas e danos.
- A precisão do contexto alinhou-se à pergunta, proporcionando uma base sólida para a resposta.

### Resposta
- A resposta gerada foi bem detalhada e consistente com o contexto jurídico recuperado.
- Incluiu explicações adicionais sobre perdas e danos, tipos e cálculo.

---

## Conclusão
O uso de metadados melhorou a **rastreamento** e a **relevância do contexto** recuperado. O modelo foi capaz de identificar o **Capítulo III – Das Perdas e Danos** e utilizá-lo para construir uma resposta clara e detalhada. Essa abordagem reforça a importância de estruturar documentos com metadados para consultas mais precisas e contextualizadas.


melhorando os metadados usando mais metadados de ML

In [155]:
import re
import pdfplumber
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

# Função para dividir o texto em chunks e gerar metadados com NMF
def split_chunks_with_ml_metadata(text, chunk_size=2000, overlap=300, page_number=None, source=None):

    current_chunk = []
    chunks = []
    current_size = 0
    chunk_index = 0
    line_start = 1

    def save_chunk_and_reset(line_end):
        nonlocal current_chunk, current_size, chunk_index, line_start
        if current_chunk:
            content = "\n".join(current_chunk)
            metadata = extract_topics_with_nmf(content)
            metadata.update({
                "linha_inicio": line_start,
                "linha_fim": line_end,
                "pagina": page_number,
                "source": source
            })
            chunks.append({
                "chunk_index": chunk_index,
                "content": content,
                "metadata": metadata
            })
            chunk_index += 1
            line_start = line_end - len(current_chunk) + 1
            current_chunk = current_chunk[-overlap:]
            current_size = sum(len(line) for line in current_chunk)

    for i, line in enumerate(text.split("\n"), start=1):
        current_chunk.append(line)
        current_size += len(line)

        if current_size >= chunk_size:
            save_chunk_and_reset(i)

    save_chunk_and_reset(len(text.split("\n")))

    return chunks

# Função para extrair tópicos com NMF
def extract_topics_with_nmf(content, n_topics=2):
    vectorizer = TfidfVectorizer(stop_words="portuguese")   
    X = vectorizer.fit_transform([content])

    nmf = NMF(n_components=n_topics, random_state=42)
    W = nmf.fit_transform(X)
    H = nmf.components_
    terms = vectorizer.get_feature_names_out()

    topics = []
    for topic_idx, topic in enumerate(H):
        top_terms = [terms[i] for i in topic.argsort()[:-5 - 1:-1]]
        topics.append(" ".join(top_terms))

    return {
        "topico_1": topics[0] if len(topics) > 0 else None,
        "topico_2": topics[1] if len(topics) > 1 else None
    }

# Função para carregar PDFs e dividir em chunks
def load_and_chunk_pdfs_with_ml_metadata(pdf_paths, chunk_size=2000, overlap=300):
    documents = []

    for pdf_path in pdf_paths:
        try:
            with pdfplumber.open(pdf_path) as pdf:
                for page_number, page in enumerate(pdf.pages, start=1):
                    text = page.extract_text()
                    if text:
                        chunks = split_chunks_with_ml_metadata(
                            text, chunk_size, overlap, page_number=page_number, source=pdf_path
                        )
                        for chunk in chunks:
                            documents.append(Document(
                                page_content=chunk["content"],
                                metadata=chunk["metadata"]
                            ))
        except Exception as e:
            pass

    return documents

# Função para criar o vetorstore
def create_vectorstore_with_metadata(documents):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore

# Função principal para todo o fluxo
def prepare_vectorstore_from_pdfs_with_ml(pdf_paths, chunk_size=2000, overlap=300):
    documents = load_and_chunk_pdfs_with_ml_metadata(pdf_paths, chunk_size, overlap)
    vectorstore = create_vectorstore_with_metadata(documents)
    return [vectorstore, documents]


In [ ]:
vectorstore_ml, documents_ml = prepare_vectorstore_from_pdfs_with_ml(pdf_files, chunk_size=2000, overlap=300)

In [148]:
def ask_question_with_history_v3(question, vectorstore, conversation_history):
    from langchain.chains import ConversationalRetrievalChain

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        return_source_documents=True  # Garante que as fontes sejam retornadas
    )

    result = qa_chain({"question": question, "chat_history": conversation_history})

    conversation_history.append((question, result["answer"]))

    metadata_list = []
    for doc in result["source_documents"]:
        metadata = doc.metadata
        metadata_list.append({

            "topico_1": metadata.get("topico_1"),
            "topico_2": metadata.get("topico_2"),
            "content": doc.page_content
        })

    return {
        "response": result["answer"],
        "context_used": metadata_list,
    }

In [ ]:



# Exemplo de uso
conversation_history = []
response1 = ask_question_with_history_v3(
    "Perdas e danos",
    vectorstore_ml,
    conversation_history
)

# Exibir resposta e contexto
print("\nResposta Gerada:")
print(response1["response"])
print("-------------------------------------------------------------------------------------------------------")
for context in response1["context_used"]:
    print(f"Tópico 1: {context.get('topico_1', 'Não encontrado')}")
    print(f"Tópico 2: {context.get('topico_2', 'Não encontrado')}")
    print("Conteúdo:")
    print(context.get("content", "Conteúdo não encontrado"))
    print("-------------------------------------------------------------------------------------------------------")




Não foi muito efetivo o uso de 